In [1]:
import gzip
import json
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix
import torch
from torch_geometric.data import Data
from torch_geometric.utils import from_scipy_sparse_matrix
from torch_geometric.transforms import RandomNodeSplit
from sklearn.model_selection import train_test_split
from torch_geometric.nn import GCNConv
from torch.nn import Linear, ModuleList
import torch.nn.functional as F
from torch.optim import Adam
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def buildBST(array, start=0, finish=-1):
    if finish < 0:
        finish = len(array)
    mid = (start + finish) // 2
    if mid-start == 1:
        ltl = start
    else:
        ltl = buildBST(array, start, mid)
    if finish-mid == 1:
        gtl = mid
    else:
        gtl = buildBST(array, mid, finish)
    return (array[mid], ltl, gtl)

def getGRCIndex(x, y, xbst, ybst):
    while isinstance(xbst, tuple):
        if x < xbst[0]:
            xbst = xbst[1]
        else:
            xbst = xbst[2]
    while isinstance(ybst, tuple):
        if y < ybst[0]:
            ybst = ybst[1]
        else:
            ybst = ybst[2]
    return ybst, xbst

def load_and_prepare_data(design_path, conn_path, congestion_path):
    # Load design data
    with gzip.open(design_path, 'rb') as f:
        design = json.loads(f.read().decode('utf-8'))

    # Load connectivity data
    conn = np.load(conn_path)
    A = coo_matrix((conn['data'], (conn['row'], conn['col'])), shape=conn['shape'])
    A = A.__mul__(A.T)
    # Load congestion data
    congestion_data = np.load(congestion_path)

    # Assuming the design includes information to create instances for processing
    instances = pd.DataFrame(design['instances'])  # Adjust as necessary based on actual design data structure

    # Build BSTs for spatial indexing
    xbst = buildBST(congestion_data['xBoundaryList'])
    ybst = buildBST(congestion_data['yBoundaryList'])

    # Calculate demand features
    demand = np.zeros(shape=(instances.shape[0],))
    for k in range(instances.shape[0]):
        xloc, yloc = instances.iloc[k]['xloc'], instances.iloc[k]['yloc']
        i, j = getGRCIndex(xloc, yloc, xbst, ybst)
        d = sum(congestion_data['demand'][lyr][i][j] for lyr in range(len(congestion_data['layerList'])))
        demand[k] = d
    instances['routing_demand'] = demand

    # Prepare node features and labels for PyTorch Geometric
    X = torch.tensor(instances[['xloc', 'yloc']].values, dtype=torch.float).to(device)  # Adjust based on actual features
    y = torch.tensor(instances['routing_demand'].values, dtype=torch.float).to(device)
    ei = from_scipy_sparse_matrix(A)
    edge_index = ei[0].to(device)
    data = Data(x=X, edge_index=edge_index, y=y).to(device)
    return data

In [4]:
from torch.optim.lr_scheduler import StepLR
def test(model, data):
    model.eval()
    with torch.no_grad():
        predictions = model(data.x, data.edge_index)
        mse_loss = torch.nn.MSELoss()
        loss = mse_loss(predictions.squeeze(), data.y.float())
    return loss.item()  # Return the loss value

# def compute_l1_loss(model, beta=0.001):
#     l1_loss = torch.tensor(0., requires_grad=True)
#     for param in model.parameters():
#         l1_loss = l1_loss + torch.norm(param, 1)
#     return beta * l1_loss

def train_and_evaluate(model, train_data, test_data, epochs=800, print_interval=10):
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        
        # Forward pass on train data
        z = model(train_data.x, train_data.edge_index)
        train_loss = criterion(z.squeeze(), train_data.y.float())
        
        # Backward pass
        train_loss.backward()
        optimizer.step()
        
        if epoch % print_interval == 0:
            # Evaluate on test data without gradient update
            test_loss = test(model, test_data)
            print(f'Epoch {epoch:>3} | Train Loss: {train_loss:.2f} | Test MSE: {test_loss:.2f}')

In [5]:
all_datasets = []
for i in range(1, 14):
    design_path = f'xbar/{i}/xbar.json.gz'
    conn_path = f'xbar/{i}/xbar_connectivity.npz'
    congestion_path = f'xbar/{i}/xbar_congestion.npz'
    graph_data = load_and_prepare_data(design_path, conn_path, congestion_path)
    all_datasets.append(graph_data)

### GCN

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Linear, BatchNorm1d, Dropout
from torch_geometric.nn import GCNConv, GATConv

class AdvancedGCNRegression(torch.nn.Module):
    def __init__(self, num_node_features, num_edge_features, conv1_out_features=16, conv2_out_features=32, conv3_out_features=32, gat_out_features=32, gat_heads=4, dropout_rate=0.7):
        super(AdvancedGCNRegression, self).__init__()
        self.conv1 = GCNConv(num_node_features, conv1_out_features)
        self.bn1 = BatchNorm1d(conv1_out_features)
        self.conv2 = GCNConv(conv1_out_features, conv2_out_features)
        self.bn2 = BatchNorm1d(conv2_out_features)
        self.conv3 = GCNConv(conv2_out_features, conv3_out_features)
        self.bn3 = BatchNorm1d(conv3_out_features)
        self.attention = GATConv(conv3_out_features, gat_out_features, heads=gat_heads, concat=True)
        self.bn_attention = BatchNorm1d(gat_out_features * gat_heads)
        self.dropout = Dropout(dropout_rate)
        self.lin = Linear(gat_out_features * gat_heads, 1)
        
    def forward(self, x, edge_index, edge_attr=None, batch=None):
        if edge_attr is not None:
            edge_features = self.edge_lin(edge_attr)
            x += edge_features[:x.size(0)]
        x = F.relu(self.bn1(self.conv1(x, edge_index)))
        x = self.dropout(x)
        x = F.relu(self.bn2(self.conv2(x, edge_index)))
        x = self.dropout(x)
        x_res = F.relu(self.bn3(self.conv3(x, edge_index)))
        x = x + x_res
        x = self.dropout(x)
        x = F.elu(self.bn_attention(self.attention(x, edge_index)))
        x = self.lin(x)
        
        return x


In [31]:
test_losses = []
models = []

for i in range(len(all_datasets)):
    print(f"LOOCV iteration {i+1}/{len(all_datasets)}: Testing on dataset {i+1}")
    model = AdvancedGCNRegression(num_node_features=all_datasets[0].x.size(1), 
                                  num_edge_features=(0 if all_datasets[0].edge_attr is None else all_datasets[0].edge_attr.size(1))
                                 ).to(device)
    for j, data in enumerate(all_datasets):
        if j != i:
            print(f"Training on dataset {j+1}")
            train_and_evaluate(model, data, all_datasets[i])
            
    model_path = f"model_{i}.pt"
    torch.save(model.state_dict(), model_path)
    models.append(model_path)

    test_loss = test(model, all_datasets[i])
    test_losses.append(test_loss)
    print(f'Test Loss for dataset {i+1} as test set: {test_loss:.2f}')
average_test_loss = sum(test_losses) / len(test_losses)
print(f'Average Test Loss after LOOCV: {average_test_loss:.2f}')

LOOCV iteration 1/13: Testing on dataset 1
Training on dataset 2
Epoch   0 | Train Loss: 1344.11 | Test MSE: 1158.11
Epoch  10 | Train Loss: 634.30 | Test MSE: 418.79
Epoch  20 | Train Loss: 106.03 | Test MSE: 215.20
Epoch  30 | Train Loss: 138.20 | Test MSE: 672.84
Epoch  40 | Train Loss: 82.26 | Test MSE: 262.67
Epoch  50 | Train Loss: 78.39 | Test MSE: 229.12
Epoch  60 | Train Loss: 72.75 | Test MSE: 287.00
Epoch  70 | Train Loss: 68.06 | Test MSE: 234.09
Epoch  80 | Train Loss: 67.22 | Test MSE: 228.77
Epoch  90 | Train Loss: 67.26 | Test MSE: 221.96
Epoch 100 | Train Loss: 65.43 | Test MSE: 220.96
Epoch 110 | Train Loss: 65.08 | Test MSE: 254.12
Epoch 120 | Train Loss: 64.43 | Test MSE: 228.05
Epoch 130 | Train Loss: 63.99 | Test MSE: 213.00
Epoch 140 | Train Loss: 61.13 | Test MSE: 218.86
Epoch 150 | Train Loss: 60.66 | Test MSE: 217.57
Epoch 160 | Train Loss: 61.54 | Test MSE: 210.73
Epoch 170 | Train Loss: 61.38 | Test MSE: 213.79
Epoch 180 | Train Loss: 61.87 | Test MSE: 188.4

TypeError: AdvancedGCNRegression.forward() missing 1 required positional argument: 'edge_index'

In [49]:
model_losses = []

for model_path in models:
    current_model_losses = []
    print(f"Evaluating {model_path}")

    current_model = AdvancedGCNRegression(num_node_features=all_datasets[0].x.size(1), 
                                          num_edge_features=(0 if all_datasets[0].edge_attr is None else all_datasets[0].edge_attr.size(1))
                                         ).to(device)
    current_model.load_state_dict(torch.load(model_path))

    for i in range(len(all_datasets)):
        loss = test(current_model, all_datasets[i])
        current_model_losses.append(loss)
        print(f'Test Loss for dataset {i+1} as test set: {loss:.2f}')

    average_loss = sum(current_model_losses) / len(current_model_losses)
    model_losses.append(average_loss)
    print(f'Average Test Loss using {model_path}: {average_loss:.2f}')

Evaluating model_0.pt
Test Loss for dataset 1 as test set: 116.45
Test Loss for dataset 2 as test set: 71.54
Test Loss for dataset 3 as test set: 92.62
Test Loss for dataset 4 as test set: 92.31
Test Loss for dataset 5 as test set: 98.26
Test Loss for dataset 6 as test set: 85.46
Test Loss for dataset 7 as test set: 51.78
Test Loss for dataset 8 as test set: 71.39
Test Loss for dataset 9 as test set: 90.03
Test Loss for dataset 10 as test set: 117.40
Test Loss for dataset 11 as test set: 56.77
Test Loss for dataset 12 as test set: 34.56
Test Loss for dataset 13 as test set: 23.65
Average Test Loss using model_0.pt: 77.09
Evaluating model_1.pt
Test Loss for dataset 1 as test set: 129.06
Test Loss for dataset 2 as test set: 73.08
Test Loss for dataset 3 as test set: 96.69
Test Loss for dataset 4 as test set: 92.83
Test Loss for dataset 5 as test set: 98.95
Test Loss for dataset 6 as test set: 87.31
Test Loss for dataset 7 as test set: 50.66
Test Loss for dataset 8 as test set: 63.50
Test

In [76]:
model_losses

[77.09371860210712,
 76.83965726999136,
 75.57783684363731,
 76.16091830913837,
 75.3321917607234,
 73.65655722984901,
 75.14592537513145,
 77.43708243736855,
 77.01727382953351,
 78.41267967224121,
 79.16501133258527,
 76.97373845027043,
 54.51044288048377]

In [75]:
min(model_losses)

54.51044288048377

In [51]:
test_losses

[116.44715118408203,
 73.08338165283203,
 93.88067626953125,
 92.7551040649414,
 100.53932189941406,
 83.89147186279297,
 48.85813522338867,
 64.96197509765625,
 88.01238250732422,
 117.33387756347656,
 53.60877227783203,
 35.911006927490234,
 34.57540512084961]

In [ ]:
# final_test_data = load_and_prepare_data('xbar/13/xbar.json.gz', 'xbar/13/xbar_connectivity.npz', 'xbar/13/xbar_congestion.npz')
# model = AdvancedGCNRegression(num_node_features=final_test_data.x.size(1), num_edge_features= (0 if final_test_data.edge_attr is None else final_test_data.edge_attr.size(1))).to(device)
# for i in range(1, 13):
#     design_path = f'xbar/{i}/xbar.json.gz'
#     conn_path = f'xbar/{i}/xbar_connectivity.npz'
#     congestion_path = f'xbar/{i}/xbar_congestion.npz'
#     graph_data = load_and_prepare_data(design_path, conn_path, congestion_path)
#     train_and_evaluate(model, graph_data, final_test_data)
# test(model, final_test_data)

### Mean Predictor

In [61]:
def baseline_model_pytorch(train_data, test_data):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    train_y = train_data.y.float().to(device)
    test_x = test_data.x.to(device)
    test_y = test_data.y.float().to(device)
    
    train_mean = train_y.mean()
    baseline_predictions = torch.full_like(test_y, train_mean)
    
    mse = torch.nn.MSELoss()
    test_loss = mse(baseline_predictions, test_y).item()
    
    return test_loss

In [63]:
test_losses = []
for i in range(len(all_datasets)):
    print(f"LOOCV iteration {i+1}/{len(all_datasets)}: Testing on dataset {i+1}")

    train_datasets = all_datasets[:i] + all_datasets[i+1:]
    test_dataset = all_datasets[i]

    all_train_data = {
        'x': torch.cat([d.x for d in train_datasets], dim=0),
        'y': torch.cat([d.y for d in train_datasets], dim=0),
    }
    
    train_data = SimpleNamespace(**all_train_data)

    test_loss = baseline_model_pytorch(train_data, test_dataset)
    test_losses.append(test_loss)
    print(f'Test Loss for dataset {i+1} as test set: {test_loss:.2f}')

average_test_loss = sum(test_losses) / len(test_losses)
print(f'Average Test Loss after LOOCV: {average_test_loss:.2f}')

LOOCV iteration 1/13: Testing on dataset 1
Test Loss for dataset 1 as test set: 32.59
LOOCV iteration 2/13: Testing on dataset 2
Test Loss for dataset 2 as test set: 136.70
LOOCV iteration 3/13: Testing on dataset 3
Test Loss for dataset 3 as test set: 59.17
LOOCV iteration 4/13: Testing on dataset 4
Test Loss for dataset 4 as test set: 58.64
LOOCV iteration 5/13: Testing on dataset 5
Test Loss for dataset 5 as test set: 57.79
LOOCV iteration 6/13: Testing on dataset 6
Test Loss for dataset 6 as test set: 59.36
LOOCV iteration 7/13: Testing on dataset 7
Test Loss for dataset 7 as test set: 45.21
LOOCV iteration 8/13: Testing on dataset 8
Test Loss for dataset 8 as test set: 41.69
LOOCV iteration 9/13: Testing on dataset 9
Test Loss for dataset 9 as test set: 63.75
LOOCV iteration 10/13: Testing on dataset 10
Test Loss for dataset 10 as test set: 63.70
LOOCV iteration 11/13: Testing on dataset 11
Test Loss for dataset 11 as test set: 38.29
LOOCV iteration 12/13: Testing on dataset 12
Te

In [ ]:
from types import SimpleNamespace
train_datasets = all_datasets[:-1]
test_dataset = all_datasets[12]

all_train_data = {
    'x': torch.cat([d.x for d in train_datasets], dim=0),
    'y': torch.cat([d.y for d in train_datasets], dim=0),
}

train_data = SimpleNamespace(**all_train_data)

test_loss = baseline_model_pytorch(train_data, test_dataset)

print(f'Test Loss when trained on datasets 1-12 and tested on dataset 13: {test_loss:.2f}')

### Random Forest Regressor

In [13]:
test_losses = []
for i in range(len(all_datasets)):
    print(f"LOOCV iteration {i+1}/{len(all_datasets)}: Testing on dataset {i+1}")
    X_train, y_train = [], []
    for j, data in enumerate(all_datasets):
        if j != i:
            for node_features in data.x:
                X_train.append(node_features.cpu().numpy())
            
            for target in data.y:
                y_train.append(target.cpu().item())

    X_train = np.array(X_train)
    y_train = np.array(y_train)

    model = RandomForestRegressor()

    model.fit(X_train, y_train)

    X_test = [node_features.cpu().numpy() for node_features in all_datasets[i].x]
    y_test = [target.cpu().item() for target in all_datasets[i].y]

    y_pred = model.predict(X_test)

    test_loss = mean_squared_error(y_test, y_pred)
    test_losses.append(test_loss)
    print(f'Test Loss for dataset {i+1} as test set: {test_loss:.2f}')

average_test_loss = sum(test_losses) / len(test_losses)
print(f'Average Test Loss after LOOCV: {average_test_loss:.2f}')

LOOCV iteration 1/13: Testing on dataset 1
Test Loss for dataset 1 as test set: 51.98
LOOCV iteration 2/13: Testing on dataset 2
Test Loss for dataset 2 as test set: 99.07
LOOCV iteration 3/13: Testing on dataset 3
Test Loss for dataset 3 as test set: 43.06
LOOCV iteration 4/13: Testing on dataset 4
Test Loss for dataset 4 as test set: 41.02
LOOCV iteration 5/13: Testing on dataset 5
Test Loss for dataset 5 as test set: 40.53
LOOCV iteration 6/13: Testing on dataset 6
Test Loss for dataset 6 as test set: 38.93
LOOCV iteration 7/13: Testing on dataset 7
Test Loss for dataset 7 as test set: 44.09
LOOCV iteration 8/13: Testing on dataset 8
Test Loss for dataset 8 as test set: 43.94
LOOCV iteration 9/13: Testing on dataset 9
Test Loss for dataset 9 as test set: 42.09
LOOCV iteration 10/13: Testing on dataset 10
Test Loss for dataset 10 as test set: 50.53
LOOCV iteration 11/13: Testing on dataset 11
Test Loss for dataset 11 as test set: 40.32
LOOCV iteration 12/13: Testing on dataset 12
Tes